<a href="https://colab.research.google.com/github/Shadidi/NLP_Chatbot/blob/main/GeographyChatbot_Kharsa_and_Hadidi_comp4992_Spring2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Trying to build a model using wiki's data.**
 

In [ ]:
# # Parsing the data file into sentences
# import nltk.data
# nltk.download('punkt')
# import numpy as np

# sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
# wikidata_Sentences = []
# # corpus = ""
# with open("Wikipedia_Geography.txt") as dfile:
#   corpus = dfile.readlines()
# for x in corpus:
#   wikidata_Sentences.append((sent_detector.tokenize(x.strip())))
# data = [item for sublist in wikidata_Sentences for item in sublist]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Vectorize the data**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorization (data):
  vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=5000, min_df=2)
  vectorizer.fit(data)
  return vectorizer.transform(data)

vec_corpus = vectorization(data)

**Representation of input or queries.**

In [ ]:
input_Sample = ["what is to the east of the Misty Mountains?", "What are SIDS?", "where is the White Mountains located?", "what makes a site preferred for buildings?", "Tell me about sea level.", "define Mental maps", "what sublayer is important to a river-bed?", "Why are mineral resource-rich Central Asian countries unstable?", "What did the 2003 Almaty Ministerial Conference acknowledge?", "What are SIDS economies based on?", "What is the “Spatial Mismatch Hypothesis”?", "What is Geocriticism?", "What contributed to the fewer autonomous outdoor play for young people?", "Why does the boundary problem occurs?", "What is the two-step floating catchment area (2SFCA)?", "What is a direct cause of land system changes?"]

vec_input = vectorization(input_Sample)

**Calculate the similarities between queries vectors and the saved sentence vectors.**

In [ ]:
# read data
import nltk
nltk.download('punkt')  # tokenizer data files
from nltk.tokenize import word_tokenize, sent_tokenize

corpus = []
with open("Wikipedia_Geography.txt") as dfile:
  for line in dfile:
    corpus.extend(sent_tokenize(line.strip().lower()))  # each line may have multiple sentences
for x in range(5):
  print(corpus[x])
# preprocessing
for idx in range(len(corpus)):
    corpus[idx] = word_tokenize(corpus[idx].strip())  # tokenization
    corpus[idx] = [token for token in corpus[idx] if len(token) > 0]  # filter the empty words
    corpus[idx] = ' '.join(corpus[idx])  # merge each line of tokens into 
for x in range(5):
  print(corpus[x])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
'''geoarchaeology''' is a multi-disciplinary approach which uses the techniques and subject matter of , ,  and other s to examine topics which inform  knowledge and thought.
geoarchaeologists study the natural physical processes that affect s such as , the formation of sites through geological processes and the effects on buried sites and  post-deposition.
geoarchaeologists' work frequently involves studying  and  as well as other geographical concepts to contribute an archaeological study.
geoarchaeologists may also use computer cartography,  (gis) and s (dem) in combination with disciplines from human and social sciences and earth sciences.
[http://sapiens.revues.org/index422.html ghilardi, m. and desruelles, s. (2008) “geoarchaeology: where human, social and earth sciences meet with technology”.
' '' geoarchaeology '' ' is a multi-disciplinary approach which uses the techni

In [ ]:
# sentence vectors
try:
  from sentence_transformers import SentenceTransformer
except:
  !pip install sentence_transformers
  from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-distilroberta-base-v1')
corpus_vectors = model.encode(corpus)


     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 2.1MB 11.2MB/s 
     |████████████████████████████████| 1.2MB 51.4MB/s 
     |████████████████████████████████| 901kB 56.0MB/s 
     |████████████████████████████████| 3.3MB 57.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=738be622c89d18acd37414e0ce3fe0da59fcc23c9d00b8d0b24d2ef163e06880
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [ ]:
# compare
from scipy.spatial import distance
import numpy as np

test_dialog_utterance = input_Sample
test_vector = model.encode(test_dialog_utterance)
test_vector = model.encode(["tell me about sea level"])
cos_sim = []

#for idx in range(len(corpus_vectors)):
#  for idx2 in range(len(test_vector)):
#    cos_sim.append(distance.cosine(test_vector[idx2], corpus_vectors[idx]))

for idx in range(len(corpus_vectors)):
   cos_sim.append(distance.cosine(test_vector.flatten(), corpus_vectors[idx]))

best_index = np.argmax(cos_sim)

print('response:', corpus[best_index])
print("end")

response: he conceives of the term as an abbreviation of `` governmental rationality '' [ 1999:174 ] .
end


**BERT Model**

In [ ]:
import torch.nn as nn
from transformers.models.auto import AutoModel
import torch as torch

NUM_CLASS = 2
class MyModel(nn.Module):
    def __init__(self, num_class): #Initilaize modules.
        super().__init__()
        self.bert_model = AutoModel.from_pretrained(bert_name)
        self.dp = nn.Dropout(.2)
        self.output = nn.Linear(self.bert_model.config.hidden_size, num_class)

    def forward(self, input_doc_ids, input_masks, token_type_ids=None): # Foward pass 
      #  [s] this is the output of the hidden layer 
        output_bert = self.bert_model(
            input_doc_ids, token_type_ids=token_type_ids,
            attention_mask=input_masks
        )
        doc_emb = self.dp(output_bert[1]) 
        predictions = self.output(doc_emb)
        return predictions # ouput (B, num_class)
bert_name = 'bert-base-uncased'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
your_model = MyModel(NUM_CLASS).to(device)
print(your_model)



MyModel(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru